# Tutoriel Torchtext

Torchtext est un qui permet de pré-traiter des données textuelles. 

En particulier il permet de : 

 - Importer des données de différents formats
 - Tokeniser les phrases en listes de mots
 - Générer un vocabulaire
 - Encoder les mots en entier
 - Initialiser le vocabulaire aléatoirement ou importer d'un embedding pré-entraîné
 - Générer des batches et réaliser le padding


## La méthode `Field`

Pour commencer, on définit un `Field`, c'est une classe qui contient des informations sur comment pré-traiter les données.

On définit deux `Field` : `TEXT` qui correspond aux commentaires et `LABEL` aux labels.

Cette classe comporte plusieurs paramètres qui sont décrits dans la 
<a href="https://torchtext.readthedocs.io/en/latest/data.html#field">doc</a>.

Ici, 
 - pour le `TEXT`:
     - les commentaires sont des données sequentielles donc `sequential=True` 
     - on la tokenisation par défaut utilise `string.split`.
     - On fixe une longueur `fix_length = 200`, pour que les commentaires soient tous de la même taille sinon par défaut torchtext fait le padding de chaque séquence en prenant la plus longue dans le batch
     - `lower=True`, pour qu'ils soient en minuscule.


 - pour `LABEL` : 
     - on a `sequential=True`
     - `use_vocab=False` car les labels sont numériques (0 : négatif et 1 : positif).



In [10]:
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator
import torch
from torchtext import data

TEXT = data.Field(sequential=True, use_vocab=True, init_token=None, eos_token=None, fix_length=None, dtype=torch.int64, 
                  preprocessing=None, postprocessing=None, lower=True, tokenize=None, 
                  tokenizer_language='en', include_lengths=False, batch_first=False, pad_token='<pad>',
                  unk_token='<unk>', pad_first=False, truncate_first=False, stop_words=None, is_target=False)

LABEL = data.Field(sequential=False, use_vocab=False, init_token=None, eos_token=None, fix_length=None, dtype=torch.int64, 
                   preprocessing=None, postprocessing=None, lower=False, tokenize=None, 
                   tokenizer_language='en', include_lengths=False,batch_first=False, pad_token='<pad>', 
                   unk_token='<unk>', pad_first=False, truncate_first=False, stop_words=None, is_target=False)

## Import des données

La classe `torchtext.data.TabularDataset.splits` permet d'importer et de séparer des données aux formats CSV, TSV ou JSON.

In [119]:
train, val, test = data.TabularDataset.splits(
        path='./data/', train='train.csv',
        validation='valid.csv', test='test.csv', format='csv', skip_header=True,
        fields=[('text', TEXT), ('label', LABEL)])

## Vocabulaire

On importe des word embeddings pré-entraînés.
On utilise GloVe qui permet de prendre un corpus de texte et transforme chaque mot dans le corpus en position dans un espace de grande dimension et tous les mots similaires sont placés ensembles. Le fonctionnemet de GloVe est expliqué <a href="https://nlp.stanford.edu/pubs/glove.pdf">ici</a>.

La méthode `build_vocab` prend deux arguments : 

 - le dataset `train`
 - les pré-entraînés "word vectors" GloVe, qui sont issus de Wikipedia 2014 + Gigaword 5
 
On peut également utiliser notre propre vectors à l'aide de la classe `vocab.Vectors`.

In [102]:
TEXT.build_vocab(train, vectors="glove.6B.100d")

In [103]:
LABEL.build_vocab(train,)

In [104]:
len(LABEL.vocab)

3

## Batch

On utilise `data.BucketIterator.splits` pour construire un itérateur pour les données de train, test et validation.  

In [120]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iter, val_iter, test_iter = data.BucketIterator.splits(
    (train, val, test), 
    batch_size = BATCH_SIZE,
    device = device)

In [ ]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")

In [121]:
import torch.nn as nn

vocab = TEXT.vocab
emb_dim = 100

NameError: name 'self' is not defined

## ReversibleField

Cette méthode permet de convertir les entiers correspondant aux mots en mots.


In [140]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embed = nn.Embedding(input_dim, embedding_dim)
        
        self.embed.weight.data.copy_(vocab.vectors)
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):

        #text = [sent len, batch size]
        
        embedded = self.embed(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

In [141]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [142]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()
model = model.to(device)
criterion = criterion.to(device)

In [143]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [144]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        predictions = model(batch.text).squeeze(1)
        loss = criterion(predictions, batch.label.float())
        acc = binary_accuracy(predictions, batch.label.float())
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [145]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():

        for batch in iterator:
            predictions = model.forward(batch.text).squeeze(1)
            loss = criterion(predictions, batch.label.float())  
            acc = binary_accuracy(predictions, batch.label.float())
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [146]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [147]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iter, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, val_iter, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

TypeError: '<' not supported between instances of 'Example' and 'Example'